# Azure Cosmos DB No SQL Vector Store

In this notebook we are going to show a quick demo of how to use AzureCosmosDBNoSqlVectorSearch to perform vector searches in LlamaIndex.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [8]:
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-azure-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install llama-index

In [17]:
import os
import json
import openai
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
#from llama_index.vector_stores.azurecosmosnosql import AzureCosmosDBNoSqlVectorSearch

# Setup Azure OpenAI

In [18]:
llm = AzureOpenAI(    
    model="AZURE_OPENAI_MODEL",
    deployment_name="AZURE_OPENAI_DEPLOYMENT_NAME",
    azure_endpoint="AZURE_OPENAI_BASE",
    api_key="AZURE_OPENAI_KEY",
    api_version="AZURE_OPENAI_VERSION",
)

embed_model = AzureOpenAIEmbedding(
    model="AZURE_OPENAI_EMBEDDING_MODEL",
    deployment_name="AZURE_OPENAI_EMBEDDING_MODEL_DEPLOYMENT_NAME",
    azure_endpoint="AZURE_OPENAI_BASE",
    api_key="AZURE_OPENAI_KEY",
    api_version="AZURE_OPENAI_VERSION",
)

In [20]:
from llama_index.core import Settings

Settings.llm=llm
Settings.embed_model=embed_model

# Loading Documents

In [25]:
import os
current_path = os.getcwd()
print(current_path)

C:\Users\t-garagundi\Documents\GitHub\llama_index\llama-index-integrations\vector_stores\llama-index-vector-stores-azurecosmosnosql


In [31]:
documents = SimpleDirectoryReader(r"/docs/docs/examples/data/paul_graham/paul_graham_essay.txt").load_data()

print("Document ID:", documents[0])

ValueError: Directory .../.../.../docs/docs/examples/data/paul_graham/paul_graham_essay.txt does not exist.